In [12]:
# https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/

# FYI, 1) Prophet's name has changed from fbprophet to prophet. So anywhere instructions say fbprophet use prophet 
# 2) it only runs on versions of python less than 3.9. If you're like me, I have 3.7 installed in PythonData kernal, so I installed Prophet there using the following instructions. 
# https://medium.com/data-folks-indonesia/installing-fbprophet-prophet-for-time-series-forecasting-in-jupyter-notebook-7de6db09f93e

In [14]:
# check prophet version
import prophet
# print version number
print('Prophet %s' % prophet.__version__)

from pandas import read_csv
from matplotlib import pyplot

from pandas import to_datetime

from prophet import Prophet

from pandas import DataFrame

import pandas as pd
import numpy as np

Prophet 1.0


In [15]:
# training percentage of dataset data

train_perc = 0.66

# daily seasonality

seasonality_b = False

# remove zeros

zeros_b = True  # True = remove zero values from dataset, False = keep zeros in dataset

In [3]:
# Load metrics from CVS files

#df_dailysteps=pd.read_csv("Resources\steps_p01.csv",index_col='Dates',parse_dates=True)

#df_steps=pd.read_csv("final_merged_all_participants/steps.csv")
#df_active_duration=pd.read_csv("final_merged_all_participants/active_duration.csv")
#df_active_heart_rate=pd.read_csv("final_merged_all_participants/active_heart_rate.csv")
#df_calories=pd.read_csv("final_merged_all_participants/calories.csv")
#df_distance=pd.read_csv("final_merged_all_participants/distance.csv")
#df_resting_heartrate=pd.read_csv("final_merged_all_participants/resting_heartrate.csv")
#df_sleep=pd.read_csv("final_merged_all_participants/sleep.csv")
#df_participant_overview=pd.read_csv("final_merged_all_participants/participant_overview.csv")

#df_sleep.rename(columns = {"Date_Of_Sleep": "Dates"}, inplace = True)

# Drop the label 'time'
#df_active_heart_rate = df_active_heart_rate.drop(columns="Time")

In [16]:
# Load metrics from Database

import psycopg2
from psycopg2 import Error
from config import db_password
from config import user

try:
    # Connect to an existing database
    connection = psycopg2.connect(user= user,
                                  password=db_password,
                                  host="ec2-3-231-82-226.compute-1.amazonaws.com",
                                  port="5432",
                                  database="derfg6csuck3l8")
    # Create a cursor to perform database operations
    cursor = connection.cursor()
    
     # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    
# 1) Executing a SQL query for participant details
    cursor.execute("SELECT * FROM participants_details;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_participant_overview = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 2) Executing a SQL query for steps details
    cursor.execute("SELECT * FROM steps ORDER BY dates;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_steps = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 3) Executing a SQL query for active_duration details
    cursor.execute("SELECT * FROM active_duration ORDER BY dates;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_active_duration = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 4) Executing a SQL query for v_avg_active_heartrate details
    cursor.execute("SELECT * FROM v_avg_active_heartrate ORDER BY dates;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_active_heart_rate = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 5) Executing a SQL query for calories details
    cursor.execute("SELECT * FROM calories ORDER BY dates;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_calories = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 6) Executing a SQL query for distance details
    cursor.execute("SELECT * FROM distance ORDER BY dates;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_distance = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")

# 7) Executing a SQL query for resting_heartrate details
    cursor.execute("SELECT * FROM resting_heartrate ORDER BY dates;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_resting_heartrate = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")
    
# 8) Executing a SQL query for sleep details
    cursor.execute("SELECT * FROM sleep ORDER BY dates;")
    
    column_names = [desc[0] for desc in cursor.description]
    # Fetch result
    record = cursor.fetchall()
    df_sleep = pd.DataFrame(record, columns = column_names)
    #print("You are connected to - ", record, "\n")
    
    
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL server information
{'user': 'yeujsrmjhunnke', 'channel_binding': 'prefer', 'dbname': 'derfg6csuck3l8', 'host': 'ec2-3-231-82-226.compute-1.amazonaws.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

PostgreSQL connection is closed


In [17]:
# keep only particpant Ids

df_participant_IDs = df_participant_overview["participant_id"]

In [18]:
def parser(df):
    df.dropna()
    df = df.drop(["participant_id"],axis=1)
    if "S.No." in df:
        df = df.drop(["S.No."],axis=1)
    df.columns = ['ds', 'y']
    df['ds']= to_datetime(df['ds'])
    
    return df

In [19]:
def evaluate_model(predictions, test, perf_mape):
    
    # calculates and returns MAPE
    
    if perf_mape:
        test = test.reset_index()
        predictions = predictions.reset_index()
        mape = round(100 * np.mean(np.abs(predictions['yhat'] - test['y'])/np.abs(test['y'])),2)  # MAPE.ipynb
    else:
        mape = "No Data"
    return mape

In [20]:
def update_output_df(output_df, forecast_df, pid, dataset):
    #consolidates output dataframes for each dataset with predictions
    temp_df = pd.DataFrame(forecast_df)
    
    # drop unneeded columns
    for col in temp_df:
        if col != "ds":
            if col != "yhat":
                temp_df = temp_df.drop([col],axis=1)
 
    
    temp_df['Participant_ID'] = pid # add participant ID
    
    if dataset == "steps":  # steps
        temp_df.rename(columns = {'ds':'Dates', 'yhat':'Steps'}, inplace = True)
        temp_df = temp_df[['Participant_ID', 'Dates', 'Steps']]
    
    elif dataset == "active_duration":  # active duration
        temp_df.rename(columns = {'ds':'Dates', 'yhat':'Active_Duration_in_mins'}, inplace = True)
        temp_df = temp_df[['Participant_ID', 'Dates', 'Active_Duration_in_mins']]
        
    elif dataset == "active_HR":  # active heart rate
        temp_df.rename(columns = {'ds':'Dates', 'yhat':'Avg_ActiveHeartRate'}, inplace = True)
        temp_df = temp_df[['Participant_ID', 'Dates', 'Avg_ActiveHeartRate']]
        
    elif dataset == "distance":  # distance
        temp_df.rename(columns = {'ds':'Dates', 'yhat':'Distance'}, inplace = True)
        temp_df = temp_df[['Participant_ID', 'Dates', 'Distance']]
        
    elif dataset == "calories":   # calories
        temp_df.rename(columns = {'ds':'Dates', 'yhat':'Calories'}, inplace = True)
        temp_df = temp_df[['Participant_ID', 'Dates', 'Calories']]
        
    elif dataset == "rest_HR":   #resting heart rate
        temp_df.rename(columns = {'ds':'Dates', 'yhat':'Resting_beats_per_min'}, inplace = True)
        temp_df = temp_df[['Participant_ID', 'Dates', 'Resting_beats_per_min']]
        
    else:    # sleep
        temp_df.rename(columns = {'ds':'Dates', 'yhat':'Minutes_Asleep'}, inplace = True)
        temp_df = temp_df[['Participant_ID', 'Dates', 'Minutes_Asleep']]
    
    
    output_df = pd.concat((output_df, temp_df), axis = 0)
    
    
    return output_df


In [21]:
def filter_output(dataset, df):
    
    # returns rows in dataframe after sepcified date
    s_date = format(dataset.ds.iat[-1], '%Y-%m-%d') # date  of last row in dataset
    filtered_df = df.loc[df['Dates'] > s_date]
    
    return filtered_df

In [22]:
# create output dataframes

active_duration_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Active_Duration_in_mins'])
steps_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Steps'])
active_heart_rate_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Avg_ActiveHeartRate'])
distance_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Distance'])
calories_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Calories'])
resting_heartrate_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Resting_beats_per_min'])
sleep_forcast_df = pd.DataFrame(columns = ['Participant_ID', 'Dates', 'Minutes_Asleep'])
mape_df = pd.DataFrame(columns = ['Participant_ID', 'steps', 'active_duration', 'active_heart_rate', 'distance', 'calories', 'resting_heartrate', 'sleep', 'Training_Percentage', 'Daily_Seasonality'])

In [23]:
#convert date fields to datetime

sleep_forcast_df['Dates'] = pd.to_datetime(sleep_forcast_df['Dates'])
active_duration_forcast_df['Dates'] = pd.to_datetime(active_duration_forcast_df['Dates'])
steps_forcast_df['Dates'] = pd.to_datetime(steps_forcast_df['Dates'])
active_heart_rate_forcast_df['Dates'] = pd.to_datetime(active_heart_rate_forcast_df['Dates'])
distance_forcast_df['Dates'] = pd.to_datetime(distance_forcast_df['Dates'])
calories_forcast_df['Dates'] = pd.to_datetime(calories_forcast_df['Dates'])
resting_heartrate_forcast_df['Dates'] = pd.to_datetime(resting_heartrate_forcast_df['Dates'])

In [24]:
for id in df_participant_IDs:
    
    print(id)
# loop through Participant IDs

    if zeros_b:
        df_steps_X = df_steps.loc[(df_steps["participant_id"] == id) & (df_steps["steps"] > 0), :]
        df_active_duration_X = df_active_duration.loc[(df_active_duration["participant_id"] == id) & (df_active_duration["active_duration_in_mins"] > 0), :]
        df_active_heart_rate_X = df_active_heart_rate.loc[(df_active_heart_rate["participant_id"] == id) & (df_active_heart_rate["avg"] > 0), :]
        df_distance_X = df_distance.loc[(df_distance["participant_id"] == id) & (df_distance["distance"] > 0), :]
        df_calories_X = df_calories.loc[(df_calories["participant_id"] == id) & (df_calories["calories"] > 0), :]
        df_resting_heartrate_X = df_resting_heartrate.loc[(df_resting_heartrate["participant_id"] == id) & (df_resting_heartrate["resting_beats_per_min"] > 0), :]
        df_sleep_X = df_sleep.loc[(df_sleep["participant_id"] == id) & (df_sleep["minutes_asleep"] > 0), :]
    else:
        df_steps_X = df_steps.loc[df_steps["participant_id"] == id, :]
        df_active_duration_X = df_active_duration.loc[df_active_duration["participant_id"] == id, :]
        df_active_heart_rate_X = df_active_heart_rate.loc[df_active_heart_rate["participant_id"] == id, :]
        df_distance_X = df_distance.loc[df_distance["participant_id"] == id, :]
        df_calories_X = df_calories.loc[df_calories["participant_id"] == id, :]
        df_resting_heartrate_X = df_resting_heartrate.loc[df_resting_heartrate["participant_id"] == id, :]
        df_sleep_X = df_sleep.loc[df_sleep["participant_id"] == id, :]
    
    steps_b = True
    active_duration_b = True
    active_heart_rate_b = True
    distance_b = True
    calories_b = True
    resting_heartrate_b = True
    sleep_b = True
    
    #test to see if dataset has any data for given participant ID
    if df_steps_X.empty:
        steps_b = False
    
    if df_active_duration_X.empty:
        active_duration_b = False
        
    if df_active_heart_rate_X.empty:
        active_heart_rate_b = False
        
    if df_distance_X.empty:
        distance_b = False
        
    if df_calories_X.empty:
        calories_b = False
        
    if df_resting_heartrate_X.empty:
        resting_heartrate_b = False
    
    if df_sleep_X.empty:
        sleep_b = False
            
    # parse data for model    
    df_steps_X = parser(df_steps_X)
    df_active_duration_X = parser(df_active_duration_X)
    df_active_heart_rate_X = parser(df_active_heart_rate_X)
    df_distance_X = parser(df_distance_X)
    df_calories_X = parser(df_calories_X)
    df_resting_heartrate_X = parser(df_resting_heartrate_X)
    df_sleep_X = parser(df_sleep_X)

# set train and test data
    train_size = int(len(df_steps_X) * abs(train_perc))
    if train_size < 3:   # Prophet has a minimum of 2 datapoints to train model
        steps_b = False
    df_steps_test, df_steps_train = df_steps_X[0:train_size], df_steps_X[train_size:]

    train_size = int(len(df_active_duration_X) * abs(train_perc))
    if train_size < 3:
        active_duration_b = False
    df_active_duration_test, df_active_duration_train = df_active_duration_X[0:train_size], df_active_duration_X[train_size:]

    train_size = int(len(df_active_heart_rate_X) * abs(train_perc))
    if train_size < 3:
        active_heart_rate_b = False
    df_active_heart_rate_test, df_active_heart_rate_train = df_active_heart_rate_X[0:train_size], df_active_heart_rate_X[train_size:]
    
    train_size = int(len(df_distance_X) * abs(train_perc))
    if train_size < 3:
        distance_b = False
    df_distance_test, df_distance_train = df_distance_X[0:train_size], df_distance_X[train_size:]
    
    train_size = int(len(df_calories_X) * abs(train_perc))
    if train_size < 3:
        calories_b = False
    df_calories_test, df_calories_train = df_calories_X[0:train_size], df_calories_X[train_size:]

    train_size = int(len(df_resting_heartrate_X) * abs(train_perc))
    if train_size < 3:
        resting_heartrate_b = False
    df_resting_heartrate_test, df_resting_heartrate_train = df_resting_heartrate_X[0:train_size], df_resting_heartrate_X[train_size:]
    
    train_size = int(len(df_sleep_X) * abs(train_perc))
    if train_size < 3:
        sleep_b = False
    df_sleep_test, df_sleep_train = df_sleep_X[0:train_size], df_sleep_X[train_size:]
    
# define the model, fit and test
    steps_test_forecast = ""
    active_duration_test_forcast = ""
    active_heart_rate_test_forcast = ""
    distance_test_forcast = ""
    calories_test_forcast = ""
    resting_heartrate_test_forcast = ""
    sleep_test_forcast = ""    
    
    if steps_b:
        model_steps = Prophet(daily_seasonality=seasonality_b)
        model_steps.fit(df_steps_train)
        steps_test_forecast = model_steps.predict(df_steps_test)
        
        future = model_steps.make_future_dataframe(periods=365)   # predict future time series
        steps_train_predict = model_steps.predict(future)
        
        steps_forcast_df = update_output_df(steps_forcast_df, steps_train_predict, id, "steps")
        steps_forcast_df = filter_output(df_steps_X, steps_forcast_df)
       
        
    if active_duration_b:
        model_active_duration = Prophet(daily_seasonality=seasonality_b)
        model_active_duration.fit(df_active_duration_train)
        active_duration_test_forcast = model_active_duration.predict(df_active_duration_test)
        
        future = model_active_duration.make_future_dataframe(periods=365) # predict future time series
        active_duration_train_predict = model_active_duration.predict(future)

        active_duration_forcast_df = update_output_df(active_duration_forcast_df, active_duration_train_predict, id, "active_duration")
        active_duration_forcast_df = filter_output(df_active_duration_X, active_duration_forcast_df)

        
    if active_heart_rate_b:
        model_active_heart_rate = Prophet(daily_seasonality=seasonality_b)
        model_active_heart_rate.fit(df_active_heart_rate_train)
        active_heart_rate_test_forcast = model_active_heart_rate.predict(df_active_heart_rate_test)
        
        future = model_active_heart_rate.make_future_dataframe(periods=365)    # predict future time series
        active_heart_rate_train_predict = model_active_heart_rate.predict(future)
        
        active_heart_rate_forcast_df = update_output_df(active_heart_rate_forcast_df, active_heart_rate_train_predict, id, "active_HR")
        active_heart_rate_forcast_df = filter_output(df_active_heart_rate_X, active_heart_rate_forcast_df)
       
    
    if distance_b:
        model_distance = Prophet(daily_seasonality=seasonality_b)
        model_distance.fit(df_distance_train)
        distance_test_forcast = model_distance.predict(df_distance_test)
        
        future = model_distance.make_future_dataframe(periods=365)     # predict future time series
        distance_train_predict = model_distance.predict(future)

        distance_forcast_df = update_output_df(distance_forcast_df, distance_train_predict, id, "distance")
        distance_forcast_df = filter_output(df_distance_X, distance_forcast_df)

        
    if calories_b:
        model_calories = Prophet(daily_seasonality=seasonality_b)
        model_calories.fit(df_calories_train)
        calories_test_forcast = model_calories.predict(df_calories_test)
        
        future = model_calories.make_future_dataframe(periods=365)    # predict future time series
        calories_train_predict = model_calories.predict(future)
        
        calories_forcast_df = update_output_df(calories_forcast_df, calories_train_predict, id, "calories")
        calories_forcast_df = filter_output(df_calories_X, calories_forcast_df)

        
    if resting_heartrate_b:
        model_resting_heartrate = Prophet(daily_seasonality=seasonality_b)
        model_resting_heartrate.fit(df_resting_heartrate_train)
        resting_heartrate_test_forcast = model_resting_heartrate.predict(df_resting_heartrate_test)
        
        future = model_resting_heartrate.make_future_dataframe(periods=365)   # predict future time series
        resting_heartrate_train_predict = model_resting_heartrate.predict(future)
        
        resting_heartrate_forcast_df = update_output_df(resting_heartrate_forcast_df, resting_heartrate_train_predict, id, "rest_HR")
        resting_heartrate_forcast_df = filter_output(df_resting_heartrate_X, resting_heartrate_forcast_df)

     
    if sleep_b:
        model_sleep = Prophet(daily_seasonality=seasonality_b)
        model_sleep.fit(df_sleep_train)
        sleep_test_forcast = model_sleep.predict(df_sleep_test)
        
        future = model_sleep.make_future_dataframe(periods=365)    # predict future time series
        sleep_train_predict = model_sleep.predict(future)
        
        sleep_forcast_df = update_output_df(sleep_forcast_df, sleep_train_predict, id, "sleep")
        sleep_forcast_df = filter_output(df_sleep_X, sleep_forcast_df)
        


    
    mape_df = mape_df.append({'Participant_ID' : id,
                    'steps' : evaluate_model(steps_test_forecast, df_steps_test, steps_b),
                    'active_duration' : evaluate_model(active_duration_test_forcast, df_active_duration_test, active_duration_b),
                    'active_heart_rate' : evaluate_model(active_heart_rate_test_forcast, df_active_heart_rate_test, active_heart_rate_b),
                    'distance' : evaluate_model(distance_test_forcast, df_distance_test, distance_b),
                    'calories' : evaluate_model(calories_test_forcast, df_calories_test, calories_b),
                    'resting_heartrate' : evaluate_model(resting_heartrate_test_forcast, df_resting_heartrate_test, resting_heartrate_b),
                    'sleep' : evaluate_model(sleep_test_forcast, df_sleep_test, sleep_b),
                    'Training_Percentage': train_perc*100,
                    'Daily_Seasonality': seasonality_b},
                    ignore_index=True)
    
    


    


    

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


p01


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 5.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


p02


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 11.


...

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 13.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


p14


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 12.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


p15


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 21.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [26]:
# write to CSV files

steps_forcast_df.to_csv('ML_Output/steps_output.csv', index=False)
active_heart_rate_forcast_df.to_csv('ML_Output/active_Duration_output.csv', index=False)
active_heart_rate_forcast_df.to_csv('ML_Output/active_HR_output.csv', index=False)
distance_forcast_df.to_csv('ML_Output/distance_output.csv', index=False)
calories_forcast_df.to_csv('ML_Output/calories_output.csv', index=False)
resting_heartrate_forcast_df.to_csv('ML_Output/resting_hr_output.csv', index=False)
sleep_forcast_df.to_csv('ML_Output/sleep_output.csv', index=False)
mape_df.to_csv('ML_Output/mape_output.csv', index=False)